### Install necessary dependencies

In [1]:
!conda install -y BeautifulSoup4
!conda install -y lxml
import pandas as pd
import html5lib
from bs4 import BeautifulSoup
import requests

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



### Get the wikipedia table and store it in My_tabel

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'html5lib')
#print(soup.prettify())
My_table = soup.find('table',{'class':'wikitable sortable'})
#print(My_table)
links = My_table.find_all('td')
#print(links)

### Fill Dataframe line  by line taking into account the requirements from coursera

In [190]:
#init Dataframe
column_names = ['Postalcode','Borough','Neighborhood']
neighborhoods = pd.DataFrame(columns=column_names)

i=0
boroughNotAssigned = False

#loop through all entries in table
for link in links:
    
    #in case of postalcode
    if( i%3 == 0):
        postalcode = str(link).split('>')[1].split('<')[0]
        
    #in case of borough
    if ( i%3 == 1):
        
        if("Not assigned" in str(link)):
            boroughNotAssigned = True
            
        if("href" in str(link)):
            borough = str(link).split('>')[2]
            borough = borough.split("<")[0]
        else:
            borough = str(link).split(">")[1].split("<")[0]
    
    #in case of neighborhood       
    if ( i%3 == 2):
        
        link = (str(link)[:-6])
            
        if("href" in str(link)):
            neighborhood = str(link).split('>')[2]
            neighborhood = neighborhood.split("<")[0]
        else:
            neighborhood = str(link).split(">")[1].split("<")[0]
         
        #if neighborhood not assigned, give it borough
        if(neighborhood == "Not assigned"):
            neighborhood = borough
         
        # if borough assgined, add entry in Dataframe
        if(boroughNotAssigned == False):
            
            if(sum(neighborhoods["Postalcode"]==postalcode)):
                neighborhoods.replace((neighborhoods[neighborhoods["Postalcode"]==postalcode].Neighborhood).values,
                                       (neighborhoods[neighborhoods["Postalcode"]==postalcode].Neighborhood).values +', ' + neighborhood,inplace=True)
            else:
                neighborhoods = neighborhoods.append({'Postalcode': postalcode,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood}, ignore_index=True)
            
        boroughNotAssigned = False
    i = i +1

In [191]:
neighborhoods.shape

(103, 3)

### include lat and long coordinates to dataframe, read from geospatial data csv

In [192]:
geocode_df=pd.read_csv("https://cocl.us/Geospatial_data")

In [193]:
geocode_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [194]:
#initialize new dimensions (lat and long)
print(neighborhoods.shape)
neighborhoods["Latitude"]=0.
neighborhoods["Longitude"]=0.
print(neighborhoods.shape)

(103, 3)
(103, 5)


In [195]:
#add coordinates line by line
for i in range(0,len(neighborhoods)):
    code = neighborhoods["Postalcode"][i]
    lat = geocode_df[geocode_df['Postal Code'] == code].Latitude.values[0]
    long = geocode_df[geocode_df['Postal Code'] == code].Longitude.values[0]
    
    neighborhoods.at[i,"Latitude"]= lat
    neighborhoods.at[i,"Longitude"]= long
    #neighborhoods["Longitude"][i] = long
    


In [196]:
neighborhoods

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
